# Capstone Project - Predicting Commercial Fishing Habits

## Overview of Process - CRISP-DM
1. Business Understanding
2. Data Understanding
3. Data Preparation
4. Modeling
5. Evaluation
6. Deployment

1. Business Understanding
According to NOAA, illegal, unreported, and unregulated fishing activities (`IUU`) violate both national and international fishing regulations. IUU is a global problem that threatens ocean ecosystems and sustainable fisheries.  It also threatens economic secuity and the natural resources that are critical to global food security. IUU also puts law-abiding fishing operations at a disadvantage. 

Illegal fishing refers to fishing activities conducted in contravention of applicable laws and regulations, including those laws and rules adopted at the regional and international level. 

Unreported fishing refers to fishing activities that are not reported or are misreported to relevant authorities in contravention of national laws and regulations or reporting procedures of a relevant regional fisheries management organization.

Unregulated fishing occurs in areas or for fish stocks for which there are no applicable conservation or management measures and where such fishing activities are conducted in a manner inconsistent with State responsibilities for the conservation of living marine resources under international law. Fishing activities are also unregulated when occurring in an RFMO-managed area and conducted by vessels without nationality, or by those flying a flag of a State or fishing entity that is not party to the RFMO in a manner that is inconsistent with the conservation measures of that RFMO. https://www.fisheries.noaa.gov/insight/understanding-illegal-unreported-and-unregulated-fishing

AIS stands for Automatic Identification System, and is used for tracking marine vessel traffic data. AIS data is collected by the US Coast Guard through an onboard safety navigation device that transmits and monitors the location and characteristics of large vessels in the US and international waters in real time. In the United States, the Coast Guard and commercial vendors collect AIS data, which can also be used for a variety of coastal planning initiatives. https://marinecadastre.gov/ais/

AIS is a maritime navigation safety communications system standardized by the international telecommunications union and adopted by the International Maritime Organization (IMO) that provides vessel information, including the vessel's identity, type, position, course, speed, navigational status and other safety-related information automatically to appropriately equipped shore stations, other ships, and aircraft; receives automatically such information from similarly fitted ships; monitors and tracks ships; and exchanges data with shore-based facilities. More information can be found here https://www.navcen.uscg.gov/?pageName=AISFAQ#1

# 2. Data Understanding

Datasets used in this process come from two primary sources: 
1. Global Fishing Watch
2. NOAA Ocean Station Data

## Global Fishing Watch Dataset
Global fishing watch dataset was sourced from `https://globalfishingwatch.org/data-download/`. 7 separate files were downloaded, each corresponding to a type of fishing vessel:
1. `drifting_longlines.csv`
2. `fixed_gear.csv`
3. `pole_and_line.csv`
4. `purse_seines.csv`
5. `trawlers.csv`
6. `trollers.csv`
7. `unknown.csv`

Each dataset contains the following columns:
* `mmsi` - anonymized vessel identifier
* `timestamp` - unix timestamp
* `distance_from_shore` - distance from shore in meters
* `distance_from_port` - distance from port in meters
* `speed` - vessel speed in knots
* `course` - vessel course
* `lat` - latitude in decimal degrees
* `long` - longitude in decimal degrees
* `source` - The training data batch. Data was prepared by GFW, Dalhousie, and a crowd sourcing campaign. False positives are marked as false_positives
* `vessel_type` - type of vessel
* `is_fishing` - label indicating fishing activity
    * `0` = not fishing
    * `>0` = fishing. Data values between 0 and 1 indicate the average score for the position if scored by multiple people
    * `-1` = no data. 
    
`is_fishing` will be our primary target variable, with other columns available to us used as featurs. 

## NOAA Ocean Station Data (OSD)
Ocean station data was sourced from the following url: `https://www.ncei.noaa.gov/access/world-ocean-database-select/dbsearch.html`.

Using the WODselect retrieval system enables a user to search World Ocean Database and new data using user-specified criteria. 

The World Ocean Database is encoded per the following documentation: `https://www.ncei.noaa.gov/data/oceans/woa/WOD/DOC/wodreadme.pdf`

As a result, downloaded files are returned in a native `.OSD` format.  To handle reading and importing of `.OSD` data, I made use of the `wodpy` package.  More information regarding `wodpy` can be found: `https://github.com/IQuOD/wodpy`.  

After successful loading of NOAA OSD, we have access to the following additional data: 
* `z`: level depths in meters
* `z_level_qc`: level depth qc flags (0 == all good)
* `z_unc`: depth uncertainty
* `t`: level temperature in Celcius
* `t_level_qc`: level temperature qc flags (0 == all good)
* `t_unc`: temperature uncertainty
* `s`: level salinities
* `s_level_qc`: level salinity qc flags (0 == all good)
* `s_unc`: salinity uncertainty
* `oxygen`: oxygen content (mL / L)
* `phosphate`: phosphate content (uM / L)
* `silicate`: silicate content (uM / L)
* `pH`: pH levels
* `p`: pressure (decibar)

NOAA OSD will be merged with vessel AIS data to add additional features  to use for classification. 

In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np